In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
batch_size = 8
block_size = 4
max_iters = 1000
learning_rate = 2e-5
eval_iters = 100
n_embed = 384
n_head = 10
n_layer = 10
dropout = 0.2

cuda


In [3]:
chars = ''
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}
encode = lambda s : [string_to_int[c] for c in s]
decode  = lambda l: "".join([int_to_string[i] for i in l])
data = torch.tensor(encode(text), dtype=torch.long)

In [5]:
# memory map for using small sippets of text from a single file of any size
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

def get_random_chunk(split):
    file_name  = 'train_split.txt' if split == 'train' else 'val_split.txt'
    with open(file_name, 'rb') as f:
        with mmap.mmap(f.fileno(), 0 , access= mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*block_size)
            
            # Seek to the random position and read the block of text
            
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)
            
            # Decode the block to a string, ignoring any invalid byte sequence
            decoded_block = block.decode('utf-8',errors = 'ignore').replace('r','')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data
            

In [6]:
# def get_batch(split):
#     data = get_random_chunk(split)
#     ix = torch.randint(len(data) - block_size, (batch_size,))
#     x = torch.stack([data[i : i+block_size] for i in ix])
#     y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#     x,y = x.to(device), y.to(device)
#     return x,y

In [7]:
class Head(nn.Module):
    """ one head of self-attention """
    
    def __init__(self, head_size) -> None:
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,x):
        # input og size (batch, time-step , channels)
        # output of size (batch, time-step, head size)
        
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        
        # compute attention scores ("affinities")
        
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5  # (B,T,hs) @ (B,hs,T) -> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)        
        wei = F.softmax(wei,dim=-1) # (B,T,T)
        wei = self.dropout(wei)
        
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B,T,T) @ (B,T,hs) -> (B,T,hs)
        return out
        

In [8]:
class MultiHeadAttention(nn.Module):
    ''' multiple heads of self-attention in parallel '''
    
    def __init__(self, num_heads,head_size) -> None:
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads,n_embed)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) 
        # (B,T,F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

In [9]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    
    def __init__(self, n_embed) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed,n_embed),
            nn.Dropout(dropout)
        )
    
    def forward(self,x):
        return self.net(x)  

In [10]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation"""
    
    def __init__(self, n_embed,n_head) -> None:
        super().__init__()
        head_size = n_embed// n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
    
    def forward(self,x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x

In [11]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        
        self.blocks = nn.Sequential(*[Block(n_embed,n_head=n_head) for _ in range(n_layer)])
        
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed,vocab_size)
        
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight , mean= 0.0, std= 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.weight)
        
        elif isinstance(module,nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
    
    def forward(self,index,targets = None):
        B,T = index.shape
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T,C)
        
        x = tok_emb + pos_emb 
        
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        
        return logits,loss
    
    def generate(self, index,max_new_tokens):
        # index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            
            index_cond = index[:,-block_size:]
            
            # get the predictions 
            logits, loss = self.forward(index_cond)
            
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            
            # applying softmax to get probabilities
            probs = F.softmax(logits,dim=-1) # (B,C)
            
            # sample from the distribution
            index_net = torch.multinomial(probs,num_samples=1) # (B,1)
            
            # append sampled index to the running sequence
            index = torch.cat((index, index_net), dim =1) # (B,T+1)
        
        return index

model = GPTLanguageModel(vocab_size)

m = model.to(device)

In [12]:
m

GPTLanguageModel(
  (token_embedding_table): Embedding(80, 384)
  (position_embedding_table): Embedding(4, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-9): 10 x Head(
            (key): Linear(in_features=384, out_features=38, bias=False)
            (query): Linear(in_features=384, out_features=38, bias=False)
            (value): Linear(in_features=384, out_features=38, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=380, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=

In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
optimizer = torch.optim.AdamW( model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter% eval_iters == 0:
        losses = estimate_loss()
        print(f"Step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    # sample a batch of data
    xb,yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


with open('model-01.pkl','wb') as f:
    pickle.dump(model,f)
print('Model Saved')

Step: 0, train loss: 4.377, val loss: 4.378
Step: 100, train loss: 3.712, val loss: 3.714
Step: 200, train loss: 3.394, val loss: 3.396
Step: 300, train loss: 3.266, val loss: 3.255
Step: 400, train loss: 3.193, val loss: 3.209
Step: 500, train loss: 3.170, val loss: 3.223
Step: 600, train loss: 3.202, val loss: 3.168
Step: 700, train loss: 3.143, val loss: 3.154
Step: 800, train loss: 3.115, val loss: 3.120
Step: 900, train loss: 3.111, val loss: 3.128
Model Saved


In [15]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello! Can you see me? ersewigih"
oges attyd o"ltte
l. ht h v (
orl  t gf woiu nNt sfR Fm ne toaxiIhg  yess r tTrtyttoleeo
